In [0]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [0]:
# Definieer wat waardes voor je k's
k1 = 4400000
k2 = 0.003
k3 = 24000
k4 = 2.2*10**-5
k5 = 0.001344833
k6 = 0
kcat = 2.1

# Definieer de concentraties op T=0 (in eerste instantie: voor inhibitie of substraat)
E0 = 10*10**-12
ES0 = 0
P0 = 0
EI0 = 0
EIi0 = 0  #(EI*)

#Definieer constanten
I = 100*10**-9
S0 = 4*10**-6
dt = 0.01 #Dit is hoe groot je simulatie tijdstappen zijn

tframe = 60 #save a frame every 60 s
tincub = 1800 #incubate for 1800 s
tsub = 1800 # Run with substrate for 1800 s

In [0]:
concentraties = np.array([E0, ES0, P0, EI0, EIi0], dtype=np.double) #matrix van concentraties

In [0]:
  S=0 # Geen S in je systeem
  I=0 # Geen I in tje systeem
                          # [E, ES, P, EI, EI*]
  tijdevolutie0 = np.array([[-k3*I-k1*S, k2+kcat, 0, k4, 0], #E = 
                           [k1*S, -k2-kcat, 0, 0, 0], #ES
                           [0, kcat, 0, 0, 0], #P
                           [k3*I, 0, 0, -k4-k5, k6], #EI
                           [0, 0, 0, k5, -k6]]) #EI*

  #Na toevoeging van S
  S= S0
  tijdevolutie1 = np.array([[-k3*I-k1*S, k2+kcat, 0, k4, 0],
                           [k1*S, -k2-kcat, 0, 0, 0],
                           [0, kcat, 0, 0, 0],
                           [k3*I, 0, 0, -k4-k5, k6],
                           [0, 0, 0, k5, -k6]])

In [0]:
tijdconc = concentraties.copy()
prev_ES = -1
prev_P = tijdconc[2]
while prev_ES < tijdconc[1]: #Stop the simulation when ES is equal or smaller
    prev_ES = tijdconc[1]
    prev_P = tijdconc[2]
    tijdconc += np.dot(tijdevolutie1*dt, tijdconc) #Doe 1 dt tijdstap
delta_P0 = tijdconc[2]-prev_P #get delta_P0      

In [0]:
tIC50 = []
Is = [100*10**-10, 100*10**-9, 100*10**-8, 100*10**-7]
for I in Is:
  # Voor toevoeging van S

  S=0 # Geen S in je systeem

                          # [E, ES, P, EI, EI*]
  tijdevolutie0 = np.array([[-k3*I-k1*S, k2+kcat, 0, k4, 0], #E = 
                           [k1*S, -k2-kcat, 0, 0, 0], #ES
                           [0, kcat, 0, 0, 0], #P
                           [k3*I, 0, 0, -k4-k5, k6], #EI
                           [0, 0, 0, k5, -k6]]) #EI*

  #Na toevoeging van S
  S= S0
  tijdevolutie1 = np.array([[-k3*I-k1*S, k2+kcat, 0, k4, 0],
                           [k1*S, -k2-kcat, 0, 0, 0],
                           [0, kcat, 0, 0, 0],
                           [k3*I, 0, 0, -k4-k5, k6],
                           [0, 0, 0, k5, -k6]])
  
  # Maak een lijst van kopien van de concentraties
  tijdconc = concentraties.copy()
  t = 0  
  prev_P = tijdconc[2]
  # Voor toevoging van S (tincub/dt stappen = 30min)
  for i in range(int(tincub/dt)):
      t+= 1
      tijdconc += np.dot(tijdevolutie0*dt, tijdconc) # Doe 1 dt tijdstap
    
  # Na toevoeging van S (tsub/dt stappen)
  ts= 0
  prev_ES = -1
  while prev_ES < tijdconc[1]: #Run until max ES
    t += 1
    ts +=1
    prev_ES = tijdconc[1]
    prev_P = tijdconc[2]
    tijdconc += np.dot(tijdevolutie1*dt, tijdconc)
  tp = 0  
  while tijdconc[2]-prev_P > 0.5*delta_P0: #Run until 
      prev_P = tijdconc[2]
      t += 1
      tp += 1
      tijdconc += np.dot(tijdevolutie1*dt, tijdconc) #Doe 1 dt tijdstap
  tIC50.append((t, ts, tp)) 

In [9]:
for t, ts, tp in tIC50:
  print (t*dt, ts*dt, tp*dt)


12462.92 0.64 10662.28
1800.66 0.66 0.0
2283.58 483.58 0.0
1917.95 117.95 0.0
